# Cause Matching 

## Data Preprocessing and Analysis

In [135]:
import pandas as pd
data = pd.read_csv('Study Title REV with Sample.csv')

In [136]:
data

,Public Title of Study,Cause1,Cause1 % Match,Cause2,Cause2 % Match,Cause3,Cause3 % Match,Cause4,Cause4 % Match,Cause5,Cause5 % Match,Cause6,Cause6 % Match,Cause7,Cause7 % Match,Cause 'n',Cause 'n' % Match,Unnamed: 17,List of Causes
0,Study on complications of diabetes and risk of...,Diabetes and kidney diseases,xx% string match,Diabetes mellitus,xx% string match,Congenital heart anomalies,xx% string match,Hypertensive heart disease,xx% string match,Ischemic heart disease,xx% string match,Non-rheumatic valvular heart disease,xx% string match,Rheumatic heart disease,xx% string match,NaN,NaN,NaN,Acne vulgaris
1,Quality and outcomes in global cancer surgery.,Bladder cancer,xx% string match,Brain and central nervous system cancer,xx% string match,Breast cancer,xx% string match,Cervical cancer,xx% string match,Colon and rectum cancer,xx% string match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acute hepatitis
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acute lymphoid leukemia
3,Study of Efficacy and Safety of Two Secukinuma...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acute myeloid leukemia
4,Amroid tablets and ointments in Piles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Age-related and other hearing loss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,A questionnaire based validation study in pati...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326,To study the Relationship Between strategies p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
327,comparison between knowledge of caregivers of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328,A Study to Evaluate the Efficacy of Shirobasti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
Titles = data['Public Title of Study']
Causes = data['List of Causes'].dropna()
data = Titles + Causes

In [138]:
len(Titles)
len(Causes)

255

Hence we have  255 causes and 330 titles to search on.
Now, we will filter the titles one by one to get to know the keywords of a title and thier properties.

Now, to preprocess the Titles we will remove the mostl commonly words (stopwords) out of it so that it doesnt affect out models accuracy. Also, we will remove the underscores, puctuations, special characters, escape sequence characters.

In [139]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(sentence):
    """
    It will take the word and covert it to its meanigfull format
    finally  final finale finalized ---> Final
    """
    sentence =  " ".join([lemmatizer.lemmatize(word)for word in sentence.split()])
    return sentence

In [140]:
import re
from gensim.parsing.preprocessing import remove_stopwords
def preprocessing(sentence):
   
   ######################### Preprocessing for new_data ##########################e
    #Converting to smaller case
    sentence = sentence.lower()
    #Stop Words Removal
    sentence = remove_stopwords(sentence)
    #line break removal
    sentence = re.sub(r"\r?\\n"," ", sentence)
    #remove special characters
    sentence = re.sub(r'\W+', ' ', sentence)
    #remove numbers
    sentence = sentence.replace('\d+', '')
    sentence = re.sub(r'\b\d+\b', ' ', sentence)
    #remove punctuation
    sentence = sentence.replace('[^\w\s]','')
    #remove underscore
    sentence = sentence.replace('_', ' ')
    #stemming documents(removing ing, ly, s)
    #remove stop words and finally stem
    sentence = lemmatize(sentence)

    return sentence

In [141]:
preprocessing(" #Prajwal is a good boy.\n. his fav number is 4")

'prajwal good boy fav number'

Applying the desired processes on the titles

In [142]:
Titles = [preprocessing(i) for i in Titles if type(i) is str]

Applying the same preprocessing on CAuses

In [143]:
Causes = [preprocessing(i) for i in Causes if type(i) is str]

In [144]:
len(Causes)

255

# Similarity model Building

### 1. Approaching with jackkard similarity

Logic
 - A title with similar words as that in the cause must be more related to the cause. Hence, since the word count in intersection matters we will be going for Jackkard similarity

Approach - 
- Take a title and a cause. compare the words that are closer to (similar to)the words in the cause. and count the Jackkard similariy between them.

In [145]:
from nltk.corpus import wordnet
def get_similars(word):
    similars = []
    similars_processed = []
    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            similars.append( preprocessing(i.name()))
    
    return similars

In [146]:
get_similars('')

[]

Function to calculate Jackkard similarity

In [147]:
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

# def main

In [158]:
final_df = pd.DataFrame()
for Cause in Causes:
    print(Cause)
    for Title in Titles:
        similarity = jaccard_similarity(Title, Cause)
        final_df.append(pd.DataFrame([Title, Cause, similarity], columns= [Title, Cause, similarity]))
        
    # print(Cause)
    # cause_similars = [j for j in [get_similars(i) for i in Cause.split(" ")]]
    # print(cause_similars)
    # break
    # cause = ' '.join( i for i in cause_similars)
    # print(type(cause))
    # for id, Title in enumerate(Titles):
    #     similarity = jaccard_similarity(cause, Title)
    #     final_df.append(pd.DataFrame([Title, Cause, similarity], columns= [Title, Cause, similarity]))
        
        
        
        
    

acne vulgaris


ValueError: Shape of passed values is (3, 1), indices imply (3, 3)